# Entrega 5.1 - Clasificador (Regresión Logística)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
A

### 1.2. Entrega
***
A

### 1.3. Estructura
***
A


## 2. Diseño
***
En esta sección se introduce el modelo implementado en el laboratorio, se describen las decisiones tomadas sobre el algoritmo y se presentan los métodos de evaluación utilizados juntos con las métricas.

### 2.1. Modelo
***
La regresión logística es un método de clasificación que utiliza la misma especificación que la regresión lineal, pero en este caso la clasificación toma valores discretos.

Se utiliza la función sigmoide a su salida ya que tiende a $0$ cuando $z$ tiende a $-\infty$, y tiende a $1$ cuando $z$ tiende a $+\infty$ y es una función diferenciable.

La función sigmoide o tambien conocida como función logística se define:

$$ h_\theta(x) = g(\theta^Tx) = \frac{1}{1+e^{-\theta^Tx}}\text{ , donde } g(z)= \frac{1}{1+e^{-z}}$$

Dado que utilizamos esta función, la función de costo usada en regresion lineal no nos sirve y debemos definir una nueva función de costo para regresion logistica. Como se vio en el teorico la función de costo en su versión vectorial es la siguiente:

$$
J(\theta) = -\frac{1}{m}\sum_{i=1}^m \left [ y^T log (h_\theta(x)) + (1 - y^T ) log (1 - h_\theta(x))\right ] 
$$

Al igual que en regresion lineal se utiliza descenso por gradiente para ajustar los pesos. La regla en su versión vectorial es la siguiente:

$$ \theta := \theta - \alpha \frac{1}{m} X^T (g(X\theta) - y)  $$

Hasta este punto, el clasificador de regresión logística funciona de manera binaria haciendo clasificaciónes verdadero o falso. Se busca extender esta idea para lograr una clasificación multiclase. Se divide el problema en $(n+1)$ problemas de clasificación binaria donde se busca el valor de $h_\theta^{(i)}(x)$ mas grande:

$$
h_\theta^{(i)} = P(y=i\ \mid x;\theta) \\
\text{predecir  para } x = \max_i (h_\theta^{(i)})
$$

Para evitar sobreajuste se aplica una regularizacion a la función de costo. Esto agrega cierta generalidad al momento de clasificar ejemplos nuevos sin quedar totalmente ajustado a los ejemplos de entrenamiento. La función de costo queda como sigue:


$$J(\theta) = - \frac{1}{m} \sum_{i=1}^m \left[ y^{(i)}\ \log (h_\theta (x^{(i)})) + (1 - y^{(i)})\ \log (1 - h_\theta(x^{(i)}))\right] + \frac{\lambda}{2m}\sum_{j=1}^n \theta_j^2$$

Esta forma de regresión se conoce también como $Ridge$ ya que utiliza el cuadrado del peso para regularizar.

La nueva regla de descenso por gradiente para cada peso:

$$ \theta_0 := \theta_0 - \alpha \frac{1}{m}\sum_{i=1}^{m}  (h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_0 $$

$$\theta_j := \theta_j - \alpha \left [ \left ( \frac{1}{m}\sum_{i=1}^{m}  (h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_j \right 
) + \frac{\lambda}{m}\theta_j  \right ] $$


### 2.2. Preprocesamiento
***
Dadas ciertas consideraciones necesarias para relizar el laboratorio, se debe preprocesar el conjunto de datos.

#### 2.2.1. Extracción de candidatos y partidos
***
Luego de levantar los datos, se filtran los candidatos que tengan menos de mil votantes. Esto da como resultado que los partidos que permanezcan van a ser solo los de aquellos candidatos que tienen más de mil votos cada uno. A modo de ejemplo, en el caso del partido colorado y el partido nacional, tenían candidatos con menos de mil votantes y con más de mil votantes. En este caso se dejan sólo los candidatos que tenían más de mil votantes.
Para implementar la extracción se usaron funcionalidades de la librería pandas.

#### 2.2.2. Reducción de dimensionalidad
***
En modo experimental, se puede realizar una reducción de dimensiones utilizando PCA. Se puede comprobar si clasificando utilizando determinada cantidad de componentes principales luego de aplicar PCA, los resultados siguen similares descartando aquellas componentes de menor varianza.
Para implementar la reducción de dimensionalidad utilizamos PCA de la librería sci-kit learn.

### 2.3. Algoritmo
***
En esta subsección se presenta como se implementó la regresión logística sobre el conjunto de datos.

#### 2.3.1. Algoritmo de entrenamiento
***
Como se vio en el teórico, la regresión logística se entrena haciendo un ajuste a sus pesos por medio de un descenso por gradiente dada la función de costo seleccionada. Se utilizó la función LogisticRegression de scikit learn para implementar la regresión.

La librería provee de distintos tipos de solver para aplicar la regresión. Las opciones disponibles son newton-cg, lbfgs,
liblinear, sag y saga. El solver liblinear es usado por defecto si no se especifica y se basa en la librería liblinear en C++. Tiene buenos resultados en conjuntos de datos con muchas dimensiones. El solver newton-cg y lbfgs son buenos cuando la funciones a aproximar son cuadráticas dada la naturaleza de sus implementaciones. El solver sag y saga usan descenso por gradiente estocástico y entre ellos se diferencian en que saga también acepta otro tipo de penalty como veremos más adelante en esta sección.

El algoritmo itera hasta la convergencia. Si la convergencia no se obtiene se puede determinar una cantidad máxima de iteraciones para obtener un resultado óptimo.

#### 2.3.2. Parámetros - Normalización
***
Dado que los atributos del corpus (o sea las respuestas a las preguntas) toman todos valores de 1 a 5, no es necesario hacer una normalización de los datos de entrada.

#### 2.3.3. Parámetros - Regularización y penalización
***
Como vimos en el teórico, la regularización es utilizada para evitar que la regresión se sobreajuste a los datos de entrenamiento. Si ocurre un sobreajuste se pierde la capacidad de generalización del algoritmo. La técnica consiste en agregar un componente de penalización a la función de costo.

La implementación de regresión logistica de scikit-learn nos provee del parametro **penalty** el cual se usa para especificar la norma utilizada en la penalización. Esta norma es luego utilizada en la regularización.

Existen 4 opciones de penalización. Se tiene 'l2', 'l1', 'elasticnet' y 'None'. El tipo 'l2' se conoce como penalizacion de Ridge. El tipo 'l1' se conoce como penalizacion de Lasso. El tipo 'elasticnet' es una combinacion de 'l1' y 'l2'.

((((PONER ALGUNA FORMULA?)))
link referencia: http://enhancedatascience.com/2017/07/04/machine-learning-explained-regularization/

Los solver newton-cg, sag y lbfgs solo soportan el penalizador 'l2'. El penalizador 'elasticnet' solo es soportado por el solver 'saga'. Si se selecciona 'None' no se aplica regularizacion. El solver liblinear no soporta la opcion 'None'.

Luego de elegido el penalizador, se debe determinar un valor para el coeficiente de regularizacion. Si se toma un valor muy grande se produce un underfitting y si el valor es muy pequeno se va a sobreajustar sobre los ejemplos de entrenamiento. La libreria provee del parametro **C**. **C** es el inverso de la fuerza de regularizacion, por lo tanto debe ser un valor positivo distinto de 0. Por defecto el valor de **C** es 1.0.

#### 2.3.4. Parámetros - Enfoque multiclase y Clasificación
***
La implementacion del algoritmo da la opcion a seleccion si realizamos una clasificacion multiclase o binaria. Para esto se provee del parametro **multi_class**. Los valores posibles son ovr, multinomial y auto. En nuestro caso utilizamos ovr que significa one versus rest. Este parametro crea internamente una regresion logistica binaria por cada clase y luego como se detallo anteriormente, se elige al de mayor valor.

#### 2.3.5. Parámetros - Convergencia e iteraciones
***
Como se mencionó anteriormente, si no se llega a una convergencia, se debe detener el algoritmo en algún momento, ya sea utilizando una tolerancia (usada por defecto en 1e-04 por la implementación de la librería) o definiendo un máximo de iteraciones. En nuestro caso decidimos limitar la cantidad de iteraciones con el parámetro **max_iter**.

#### 2.3.6. Resumen
***
En resumen, para nuestra implementación de regresión logística usando la librería sci-kit learn vamos a variar los siguientes parámetros del modelo:

- **solver**: Los valores a variar son newton-cg, lbfgs,liblinear, sag y saga.
- **penalty**: Los valores a variar son l2, l1 y None. 
- **max_iter**: Este valor por defecto probamos con 1000 iteraciones.
- **C**: Este valor es el inverso del regularization strength. Como se mencionó debe ser un valor positivo mayor que cero.

Para el parámetro **multi_class** se fija el valor 'ovr' (one-versus-rest) dado que buscamos una clasificación multiclase compuesta por varios clasificadores binarios donde cada uno clasifica su clase y luego se decide según el que devuelva un valor máximo entre ellos.

### 2.4. Evaluación
***
En esta subsección se comentan métodos de evaluación utilizados y métricas consideradas.

#### 2.4.1. Métodos
***
En este laboratorio se usaron dos métodos de evaluación. Primero el método de validación normal, partiendo el conjunto de datos en 2, donde por un lado nos queda un conjunto de datos para entrenar y por otro un subconjunto para testear el algoritmo. Se realiza una partición 80/20, entrenamiento y test, respectivamente.

Para implementar la validación normal, sci-kit learn provee de una función para separar el conjunto de datos en entrenamiento y test. La función es train_test_split y se le pasan por parámetro el conjunto de entrenamiento a separar y opcionalmente algunos parámetros que si no son ingresados, se toma sus valores por defecto. Entre ellos tenemos **test_size** siendo un valor entre 0 y 1 el cual determina la proporción del conjunto de datos a usar como test. Otro parámetro es **train_size** que si no se lo ingresa, se define como **1 - test_size**. Por defecto **test_size** vale 0.25. Otro parámetro útil es **shuffle**, el cual antes de separar desordena el conjunto de datos aleatoriamente (por defecto es True).

El otro método de evaluación es la validación cruzada. Se denomina como **validación cruzada** a la partición del conjunto $D$ en $k$ particiones $T_i$ y $E_i : i \in [1..k]$,  utilizando una proporción de $\frac{k-1}{k}$ y $\frac{1}{k}$ ejemplos para $T_i$ y $E_i$ respectivamente.

Esta estrategia es más robusta que la **validación normal** anteriormente mencionada, sin embargo tiene el problema de tener un mayor orden computacional, dado que tiene que correr $k$ entrenamientos en lugar de uno.

En este caso, sci-kit learn provee de una función para implementar la validacion cruzada (**cross_validate**). A la función se le pasa el modelo (en nuestro caso la LogisticRegression), el conjunto de datos sin contener el valor objetivo (la clase de cada ejemplo de entrenamiento), la clasificación de cada ejemplo del conjunto de entrenamiento y se configura el parámetro **cv** para indicar el valor del **k** para realizar las particiones.

En la experimentación se decidió probar con $k=5$ y #k=10#.

#### 2.4.2. Métricas
***
Se utilizaron múltiples métricas para determinar la "calidad" o "eficacia" de los clasificadores entrenados. A continuación, una breve descripción de las mismas:

##### 2.4.2.1 Accuracy
***
La **accuracy** o **exactitud** hace referencia a la cantidad de aciertos que el clasificador tuvo en relación al total de clasificaciones que realizó durante la evaluación. Para definir **accuracy**, se utilizan las siguientes nociones:
- Sea $D$ el conjunto de datos, $C_D$ el conjunto de posibles clasificaciones en $D$ y $|C_D| = n$
- Sea $t_i : i \in [1..n]$ la cantidad de clasificaciones para la clase $i$ que fueron correctas
- Sea $f_i : i \in [1..n]$ la cantidad de clasificaciones para la clase $i$ que fueron incorrectas

$$ accuracy = \frac{\sum_{i=1}^{n} t_i}{\sum_{i=1}^{n} t_i + f_i}$$

Se observa que **accuracy** no es una muy buena medida de performance, ya que depende mucho del conjunto de datos que se evalúe, pudiendo dar buenos resultados a un clasificador malo. Teniendo esto en cuenta, se utilizó esta métrica solo para tener una noción general del desempeño de un clasificador, dandole más peso a las métricas definidas a continuación.

##### 2.4.2.2 Matriz de Confusión
***
La **matriz de confusión** es una matriz que permite almacenar todas las clasificaciones realizadas en una evaluación, basándose en cada resultado y su respectivo valor original. La estructura de la matriz es la siguiente:

- Sea $D =$ Conjunto de datos
- Sea $C_D =$ Conjunto de clasificaciones para $d \in D$ 
- Sea $|C_D| = n$ cantidad de posibles clasificaciones en $C_D$
- Sea $M$ la matriz de confusión para una evaluación definida como:

$$M = \begin{bmatrix}
    x_{11} & x_{12} & \dots  & x_{1n} \\
    x_{21} & x_{22} & \dots  & x_{2n} \\
    \vdots & \vdots & \ddots & \vdots \\
    x_{n1} & x_{n2} & \dots  & x_{nn}
\end{bmatrix}$$

- Entonces $x_{ij} \in M : i,j \in [1..n]$ representa la cantidad de clasificaciones que originalmente eran **clase $i$** y que fueron clasificadas en la evaluación como **clase $j$**.

De esta forma, en una matriz cuadrada de dimensión $n$, se pueden centralizar todas las clasificaciones realizadas y su relación con respecto a cada clase.

En sí misma, la matriz de confusión no representa una métrica sino una herramienta para calcular otras métricas, las cuales serán detalladas a continuación. De todas formas, una mirada superficial a una matriz de confusión de tamaño razonable puede aportar información general sobre el desempeño del clasificador, (esto es detallado en la siguiente subsección).

##### 2.4.2.3 Precision, Recall, F-measure
***
Usamos tres medidas que, si bien siguen una fórmula similar a la de **accuracy**, son mucho menos sensibles al ruido en el conjunto de datos $D$, ya que operan **por clase**. Antes de detallar cada una, se repasan ciertas nociones básicas utilizadas en la definición de cada métrica.

Retomando el punto anterior, la matriz de confusión permite calcular para cada clase $c_i$, cuatro valores indispensables:
- Los **verdaderos positivos** en la clase $c_i$ (denotados como $t_i$), son aquellos ejemplos que originalmente son clase $c_i$ y fueron clasificados correctamente en la evaluación como clase $c_i$. Para calcularlos utilizando la matriz de confusión, simplemente hay que tomar la celda $M(i,i)$ para la clase $c_i$.

$$ t_i = M(i,i) $$

- Los **falsos positivos** en la clase $c_i$ (denotados como $f_i$), son aquellos ejemplos que originalmente son clase $c_j : j \neq i$ y fueron clasificados incorrectamente en la evaluación como clase $c_i$. Para calcularlos utilizando la matriz de confusión, simplemente hay que tomar la fila $M_{(i,)}$ para la clase $c_i$, y sumar todos sus valores (excepto la celda $M(i,i)$, que corresponde a los verdaderos positivos).

$$ f_i = \sum_{j=1, j \neq i}^{n} M(i,j) $$

- Los **verdaderos negativos** en la clase $c_i$ (denotados como $\bar{t_i}$), son aquellos ejemplos que originalmente no son clase $c_i$ y fueron clasificados en la evaluación como clase $c_j : j \neq i$. Cabe destacar que la clasificación como $c_j$ puede ser correcta o no, lo importante es que sea consistente en relación a $c_i$. Para calcularlos utilizando la matriz de confusión, simplemente hay que tomar la matriz adjunta a los verdaderos positivos de $c_i$ y sumar todas sus entradas. Dicha matriz se define como $M_{ii}$ y contiene todas las entradas de $M$ excepto la fila $i$ y la columna $i$.

$$ \bar{t_i} = \sum_{j=1}^{n-1} \sum_{k=1}^{n-1} M_{ii}(j,k) $$

- Los **falsos negativos** en la clase $c_i$ (denotados como $\bar{f_i}$), son aquellos ejemplos que originalmente son clase $c_i$ y fueron clasificados incorrectamente en la evaluación como clase $c_j : j \neq i$. Para calcularlos utilizando la matriz de confusión, simplemente hay que tomar la columna $M_{(,i)}$ para la clase $c_i$, y sumar todos sus valores (excepto la celda $M(i,i)$, que corresponde a los verdaderos positivos).

$$ \bar{f_i} = \sum_{j=1, j \neq i}^{n} M(j,i) $$

Teniendo en cuenta dichas definiciones, se pasa a definir las métricas utilizadas:
- **Precision**
> - La **precision** o precisión de una **clase $c_i$** hace referencia a la cantidad de clasificaciones correctas para dicha clase, en relación a la cantidad de clasificaciones totales (correctas e incorrectas) para dicha clase.
> - La precision permite determinar "que tan bien se clasifica una clase". En otras palabras, que tan poco se equivoca el clasificador en marcar un ejemplo como cierta clase.
> - Oscila entre 0 y 1, siendo mejor cuanto **mayor** es.
> - Se calcula como:
> $$ precision(c_i) = \frac{t_i}{t_i + f_i}$$

- **Recall**
> - La **recall** o recuperación de una **clase $c_i$** hace referencia a la cantidad de clasificaciones correctas para dicha clase, en relación a la cantidad de clasificaciones originales para dicha clase.
> - La recall permite determinar "que tan bien se clasifican elementos de otras clases respecto a una clase". En otras palabras, que tan poco se equivoca el clasificador en marcar un ejemplo de otras clases como cierta clase.
> - Oscila entre 0 y 1, siendo mejor cuanto **mayor** es.
> - Se calcula como:
> $$ recall(c_i) = \frac{t_i}{t_i + \bar{f_i}}$$

- **F-measure**
> - La **F-measure** o medida-F de una **clase $c_i$** es una medida que relaciona **precision** y **recall**, oficiando de alguna forma como un representante de cierta clase.
> - La F-measure permite determinar "que tan bien se clasifica una clase y que tan poco se falla a la vez". En otras palabras, es una relación proporcional entre la precision y la recall y por tanto es una buena métrica para determinar el desempeño del clasificador respecto a una clase.
> - Oscila entre 0 y 1, siendo mejor cuanto **mayor** es.
> - Se calcula como:
> $$ Fmeasure(c_i) = \frac{2 precision(c_i) recall(c_i)}{precision(c_i) + recall(c_i)}$$

Si bien estas medidas ofrecen información más enriquecedora en comparación a las anteriormente mencionadas, cuentan con el "defecto" de aplicar por clase y no dar una medida general del clasificador. Para conjuntos $D$ con pocas clasificaciones en $C_D$ esto no es un gran problema, pero complica la comparación al aumentar la cantidad de clases.


## 3. Experimentación
***
A

### 3.1. Metodología
***
A

1. **Evaluación del corpus:** A.
<br><br>
2. **Configuraciones paramétricas:** A.
<br><br>
3. **Elección y comparación de representantes:** A.
<br><br>
4. **Comparación de clasificadores según partido:** A.

### 3.2. Evaluación del corpus
***
A


In [2]:
# Script que grafica métricas del corpus
# Se grafica el explained variance ratio para ver mejor k de PCA
# Se grafica la distribución de candidatos y partidos con más de 1000 votos

### 3.3. Configuración paramétrica
***
A

#### 3.3.1. Evaluaciones
***
A

#### 3.3.2. Resultados
***
A


In [1]:
# Script que dada una configuración paramétrica, la entrena y evalúa
# Evaluación muestra accuracy, metricas, matriz de confusión

### 3.4. Elección y comparación de representantes
***
A

#### 3.4.1. Análisis cuantitativo
***
A


In [ ]:
# Script que evalúa con cross validation cada configuración paramétrica y las devuelve ordenadas por accuracy
# Devuelve los 4 mejores que son: Candidatos sin PCA, Partidos sin PCA, Candidatos con PCA y Partidos con PCA

#### 3.4.2. Análisis cualitativo
***
A


In [ ]:
# Script que grafica métricas de los módulos elegidos con el anterior script
# Se grafica en barras las metricas, en mapa de calor las matrices de confusión

#### 3.4.3. Representantes elegidos
***
A


### 3.5. Comparación de clasificadores según partido
***
A


In [ ]:
# Script que evalúa con cross validation a los representantes de candidatos y sustituye sus candidatos por partidos,
# comparando rendimiento con representantes de partidos
# Tambien grafica métricas en barras, mapa de calor de matrices de confusión

## 4. Conclusiones
***

#### 4.1. Respecto a los Datos
***
- A

#### 4.2. Respecto a los Parámetros
***
- A

#### 4.3. Respecto a los Resultados
***
- A

## 5. Referencias
***
[1] **Teoría analítica de la política** - *Melvin J. Hinich y Michael C. Munger* (2003)

[2] **Un centro vacío de candidatos: evaluando modelos espaciales para las elecciones presidenciales en Uruguay** - *Juan Andrés Moraes y Diego Luján (2016)*

[3] **The Political Spectrum: A Bi-Dimensional Approach** - *Maurice Bryson y William McDill* (1968)